# User Study Results

This notebook will be used to analyse the results from the user study, and generate associated graphical material.

In [ ]:
# Import data into dataframe for easy use.

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme("paper")
import os
import plot_likert
import numpy as np

RESULTS_PATH = os.path.join(os.path.abspath(""), "user_study_results.csv")
results_raw = pd.read_csv(RESULTS_PATH)
# All users consented to data being used. Drop auxiliary questions
results_raw = results_raw.drop(results_raw.columns[[0,1,2,3,-1,-2]], axis = 1)

results_task_1 = results_raw.drop(results_raw.columns[[2,5,8,11,14, 15,16,17,18,19]], axis = 1)
results_global = results_raw.iloc[:,[15,16,17]]

## Visualise Gloabl Results

In [ ]:
fig_1, (ax1) = plt.subplots(figsize = (8,5))
bar = plot_likert.plot_likert(results_global.astype(int), plot_scale=[1,2,3,4,5], legend=None, plot_percentage=True, bar_labels=True, bar_labels_color="snow", ax=ax1)
handles, labels = ax1.get_legend_handles_labels()
fig_1.legend(handles, plot_likert.scales.agree, bbox_to_anchor=(1.11, .85))
ax1.grid(True)
ax1.set_xticklabels([])
manual_counts = ((np.array([0,0,0,1,1,2,3,1,1,8,6,10,7,11,6]))/19  * 100).round(2)
str_out = []
for i in range(len(manual_counts)):
    if manual_counts[i] == 0:
        str_out.append("")
    else:
        str_out.append(str(manual_counts[i]))
for idx, text_item in enumerate(bar.texts):
    text_item.set_text(str_out[idx])
ax1.set_xlabel("Percentage of responses (%)")
fig_1.savefig(os.path.join(os.path.abspath(""), "generated_figures", "global_results.pdf"), format="pdf",  bbox_inches = "tight")

## Task 1: Fleiss Kappa and Bar Plots

In [ ]:
results_task_1
results_task_1.fillna(0, inplace=True)
count = 0
before_yes = []
before_no = []
after_yes = []
after_no = []
for column in results_task_1.columns:
    # Prevent NaN after reload.
    results_task_1[column] = results_task_1[column].map({"Yes":1, "No":0, 0:0, 1:1})
    yes_score = results_task_1[column].to_numpy().sum()
    no_score = 19 - yes_score
    if count % 2 == 0:
        before_yes.append(yes_score)
        before_no.append(no_score)
    else:
        after_no.append(no_score)
        after_yes.append(yes_score)
    count += 1

tweet_no = 5
ind = np.arange(tweet_no)
width = 0.35 
fig_2, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5), sharey=True)
before_yes_leg = ax1.bar(ind, before_yes, width)
before_no_leg = ax1.bar(ind + width, before_no, width)
import matplotlib.ticker as ticker
for axis in [ax1.yaxis]:
    axis.set_major_locator(ticker.MaxNLocator(integer=True))
ax1.legend([before_yes_leg, before_no_leg], labels = ["Yes", "No"])
ax1.set_ylabel("Frequency")
ax1.set_title("Did you have reason to doubt the tweet a priori?")
ax1.set_xticklabels(["","Tweet 1", "Tweet 2", "Tweet 3", "Tweet 4" , "Tweet 5"], rotation = 30)

after_yes_leg = ax2.bar(ind, after_yes, width)
after_no_leg = ax2.bar(ind + width, after_no, width)
ax2.legend([after_yes_leg, after_no_leg], labels = ["Yes", "No"])
ax2.set_title("Did the browser extension support your experience?")
ax2.set_xticklabels(["","Tweet 1", "Tweet 2", "Tweet 3", "Tweet 4" , "Tweet 5"], rotation = 30)
for axis in [ax2.yaxis]:
    axis.set_major_locator(ticker.MaxNLocator(integer=True))
fig_2.tight_layout()
fig_2.savefig(os.path.join(os.path.abspath(""), "generated_figures", "task1_results.pdf"), format="pdf",  bbox_inches = "tight")

In [ ]:
arr = results_task_1.to_numpy()
from statsmodels.stats import inter_rater as irr
dats, cats = irr.aggregate_raters(arr[:,1::2].transpose())
print(arr[:,1::2].transpose())
print(dats)
print(irr.fleiss_kappa(dats, method="rand"))

dats, cats = irr.aggregate_raters(arr.transpose())
print(arr.transpose())
print(dats)
print(irr.fleiss_kappa(dats, method="rand"))

dats, cats = irr.aggregate_raters(arr[:,0::2].transpose())
print(arr[:,0::2].transpose())
print(dats)
print(irr.fleiss_kappa(dats, method="rand"))
